In [1]:
import pandas as pd
import numpy as np

# Import Data

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sample = pd.read_csv('sample_submission.csv')
df_train.head()

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km)
0,1,FOLD,PICKUP TRUCK - STANDARD,3.5,6.0,A6,X,8.64 liters per 100 km,6.14 L/100 km,37.59 mpg Imp.,306.0
1,2,CHEVO,PICKUP TRUCK - STANDARD,5.3,NaN,A6,E,27.27 liters per 100 km,30.76 liters per 100 km,28.84 L/100 km,283.0
2,3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,0.17 L/10km,2.03 liters per 100 km,zero,329.0
3,4,KIO,SUV - SMALL,NaN,4.0,AS6,X,6.52 km per L,1.59 L/10km,NaN,270.0
4,5,BARUSU,MINICOMPACT,3.0,6.0,unspecified,Z,20.15 L/100km,0.60 L/10km,1.38 L/10km,193.0


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137141 entries, 0 to 137140
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Id                     137141 non-null  int64  
 1   Make                   137141 non-null  object 
 2   Vehicle Class          135816 non-null  object 
 3   Engine Size(L)         120522 non-null  object 
 4   Cylinders              122986 non-null  object 
 5   Transmission           134536 non-null  object 
 6   Fuel Type              135286 non-null  object 
 7   Fuel Consumption City  134719 non-null  object 
 8   Fuel Consumption Hwy   134573 non-null  object 
 9   Fuel Consumption Comb  134559 non-null  object 
 10  CO2 Emissions(g/km)    137141 non-null  float64
dtypes: float64(1), int64(1), object(9)
memory usage: 11.5+ MB


# Data Cleaning

In [4]:
check_null = df_train.isnull()

for i in check_null.columns.tolist():
    print(check_null[i].value_counts())
    print('')

False    137141
Name: Id, dtype: int64

False    137141
Name: Make, dtype: int64

False    135816
True       1325
Name: Vehicle Class, dtype: int64

False    120522
True      16619
Name: Engine Size(L), dtype: int64

False    122986
True      14155
Name: Cylinders, dtype: int64

False    134536
True       2605
Name: Transmission, dtype: int64

False    135286
True       1855
Name: Fuel Type, dtype: int64

False    134719
True       2422
Name: Fuel Consumption City, dtype: int64

False    134573
True       2568
Name: Fuel Consumption Hwy, dtype: int64

False    134559
True       2582
Name: Fuel Consumption Comb, dtype: int64

False    137141
Name: CO2 Emissions(g/km), dtype: int64



In [5]:
df_train.describe(include='object').T

,count,unique,top,freq
Make,137141,21,FOLD,18081
Vehicle Class,135816,24,SUV - SMALL,27799
Engine Size(L),120522,50,2.0,21216
Cylinders,122986,15,4.0,66618
Transmission,134536,33,AS6,28926
Fuel Type,135286,13,X,83001
Fuel Consumption City,134719,33648,0,639
Fuel Consumption Hwy,134573,34963,zero,672
Fuel Consumption Comb,134559,26157,not-available,695


In [6]:
df_train['numeric_FuelCity'] = df_train['Fuel Consumption City'].str.extract(r'([\d.]+)').astype(float)
df_train['MeasureUnit_FuelCity'] = df_train['Fuel Consumption City'].str.extract(r'\s(.+)')

df_train.sample(3)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity
85189,85190,TOYOTI,MINICOMPACT,2.5,4.0,M6,Z,12.79 L/100km,0.80 L/10km,9.42 km per L,243.0,12.79,L/100km
102686,102687,JIPU,SUV - SMALL,2.4,4.0,A9,X,0.79 L/10km,11.21 liters per 100 km,9.37 L/100 km,224.0,0.79,L/10km
543,544,JIPU,SUV - SMALL,NaN,4.0,M6,X,28.95 mpg Imp.,9.39 L/100km,0.96 L/10km,210.0,28.95,mpg Imp.


In [7]:
df_train.head(3)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity
0,1,FOLD,PICKUP TRUCK - STANDARD,3.5,6.0,A6,X,8.64 liters per 100 km,6.14 L/100 km,37.59 mpg Imp.,306.0,8.64,liters per 100 km
1,2,CHEVO,PICKUP TRUCK - STANDARD,5.3,NaN,A6,E,27.27 liters per 100 km,30.76 liters per 100 km,28.84 L/100 km,283.0,27.27,liters per 100 km
2,3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,0.17 L/10km,2.03 liters per 100 km,zero,329.0,0.17,L/10km


## General

In [8]:
df_train['Fuel Consumption City'].value_counts()

0                  639
not-available      624
-9999              617
-1                 604
zero               599
                  ... 
75.92 km per L       1
46.42 MPG (AS)       1
181.27 MPG (AS)      1
58.68 mpg Imp.       1
33.71 L/100 km       1
Name: Fuel Consumption City, Length: 33648, dtype: int64

In [9]:
df_train['Fuel Consumption Hwy'].value_counts()

zero                       672
9999                       669
-9999                      660
not-available              647
-1                         644
                          ... 
65.19 mpg Imp.               1
85.28 km/L                   1
133.13 km/L                  1
33.92 liters per 100 km      1
1275.84 mpg Imp.             1
Name: Fuel Consumption Hwy, Length: 34963, dtype: int64

In [10]:
df_train['Fuel Consumption Comb'].value_counts()

not-available      695
zero               681
-9999              665
-1                 637
9999               617
                  ... 
7.80 mpg Imp.        1
118.13 mpg Imp.      1
51.68 MPG (AS)       1
69.35 MPG (AS)       1
32.27 km per L       1
Name: Fuel Consumption Comb, Length: 26157, dtype: int64

In [11]:
df_train[df_train['Fuel Consumption Comb'] == '-9999']

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity
9,10,FOLKSWA,COMPACT,2.0,4.0,AS6,X,43.84 km per L,18.13 L/100 km,-9999,202.000000,43.84,km per L
42,43,FOLD,PICKUP TRUCK - STANDARD,3.5,6.0,AS6,X,14.09 L/100 km,6.73 L/100 km,-9999,282.000000,14.09,L/100 km
349,350,JIPU,SUV - SMALL,na,6.0,M5,X,7.88 mpg Imp.,16.98 MPG (AS),-9999,215.000000,7.88,mpg Imp.
732,733,MITSU,SUV - SMALL,2.4,4.0,AV6,X,8.11 km per L,5.60 L/100km,-9999,216.000000,8.11,km per L
1411,1412,NIRRAN,MID-SIZE,2.5,4.0,M6,X,52.45 mpg Imp.,50.74 km/L,-9999,197.000000,52.45,mpg Imp.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135362,135363,LAMBOGI,TWO-SEATER,NaN,10.0,AM7,Z,12.87 L/100 km,5.32 L/100km,-9999,615.131491,12.87,L/100 km
135373,135374,LECUS,SUV - SMALL,2.0,4.0,AS6,Z,10.62 MPG (AS),35.44 MPG (AS),-9999,216.000000,10.62,MPG (AS)
135877,135878,CADILUXE,MID-SIZE,NaN,8.0,AS8,Z,18.69 km per L,12.33 liters per 100 km,-9999,408.000000,18.69,km per L
136152,136153,CHEVO,COMPACT,1.8,4.0,AS6,X,0.26 L/100km,22.72 MPG (AS),-9999,199.000000,0.26,L/100km


In [12]:
df_train[df_train['Fuel Consumption Hwy'] == '-9999']

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity
50,51,LECUS,SUBCOMPACT,3.5,6.0,AS6,Z,4.03 MPG (AS),-9999,5.69 MPG (AS),259.0,4.03,MPG (AS)
99,100,CHEVO,VAN - PASSENGER,4.8,8.0,A6,X,4.32 km per L,-9999,22.70 liters per 100 km,374.0,4.32,km per L
389,390,LAMBOGI,TWO-SEATER,5.2,10.0,AM7,Z,52.49 MPG (AS),-9999,20.79 km per L,349.0,52.49,MPG (AS)
733,734,CHEVO,SUV - SMALL,NaN,4.0,A6,X,18.57 L/100 km,-9999,7.63 km/L,207.0,18.57,L/100 km
773,774,DOGE,FULL-SIZE,3.6,6.0,A5,X,1.08 L/10km,-9999,1.07 L/10km,276.0,1.08,L/10km
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136563,136564,MATSUDA,MID-SIZE,2.0,4.0,M6,X,15.63 L/100 km,-9999,8.69 L/100 km,177.0,15.63,L/100 km
136820,136821,FOLKSWA,MID-SIZE,missing,6.0,AS6,X,16.91 L/100km,-9999,3.89 km/L,199.0,16.91,L/100km
137016,137017,BARUSU,COMPACT,2.5,4.0,M6,Z,22.11 liters per 100 km,-9999,14.65 L/100 km,287.0,22.11,liters per 100 km
137062,137063,BMV,TWO-SEATER,3.0,NaN,M6,Z,29.64 L/100 km,-9999,17.95 L/100 km,255.0,29.64,L/100 km


In [19]:
df_train[df_train['Fuel Consumption City'] == '-1']

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity
33,34,KIO,COMPACT,2.0,4.0,AS6,X,-1,14.88 km per L,-1,213.000000,1.0,NaN
203,204,BARUSU,SUV - SMALL,2.0,4.0,AV8,NaN,-1,17.43 MPG (AS),0.89 L/10km,198.000000,1.0,NaN
274,275,CHEVO,SUV - SMALL,2.5,4.0,AS6,X,-1,0.33 L/10km,0.69 L/10km,183.000000,1.0,NaN
420,421,NIRRAN,PICKUP TRUCK - SMALL,2.5,4.0,A5,X,-1,13.71 km per L,7.44 km per L,303.000000,1.0,NaN
567,568,BMV,SUBCOMPACT,2.0,4.0,M6,Z,-1,4.01 L/100km,49.73 km/L,207.000000,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135088,135089,JIPU,SUV - SMALL,1.4,NaN,M6,X,-1,52.44 MPG (AS),0.82 L/10km,290.435201,1.0,NaN
135875,135876,CHEVO,PICKUP TRUCK - STANDARD,5.3,8.0,A6,X,-1,8.93 km per L,12.83 liters per 100 km,289.000000,1.0,NaN
136025,136026,FOLD,SUBCOMPACT,NaN,4.0,not-recorded,X,-1,1.19 L/10km,0,194.000000,1.0,NaN
136371,136372,FIAR,STATION WAGON - SMALL,1.4,4.0,AM6,X,-1,9.07 L/100 km,14.29 L/100km,203.000000,1.0,NaN


In [18]:
df_train[df_train['Fuel Consumption City'] == 'not-available']

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity
344,345,KIO,STATION WAGON - SMALL,NaN,4.0,AS6,X,not-available,11.26 L/100 km,zero,213.0,NaN,NaN
434,435,TOYOTI,PICKUP TRUCK - SMALL,3.5,6.0,AS6,X,not-available,12.89 mpg Imp.,2.28 L/10km,273.0,NaN,NaN
467,468,DOGE,MID-SIZE,6.4,NaN,M6,Z,not-available,10.75 L/100km,5.12 km/L,311.0,NaN,NaN
637,638,JIPU,SUV - SMALL,2.4,4.0,A9,X,not-available,25.05 mpg Imp.,6.52 km/L,234.0,NaN,NaN
831,832,BMV,MID-SIZE,3.0,NaN,AS8,Z,not-available,0.26 L/10km,11.22 km per L,223.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135530,135531,BMV,COMPACT,2.0,4.0,AS8,Z,not-available,14.86 L/100km,7.49 L/100km,209.0,NaN,NaN
135534,135535,TOLVO,SUV - SMALL,3.2,6.0,AS6,unspecified,not-available,57.16 MPG (AS),33.10 mpg Imp.,244.0,NaN,NaN
136019,136020,CHEVO,FULL-SIZE,3.6,6.0,AS6,X,not-available,11.12 km/L,11.81 MPG (AS),246.0,NaN,NaN
136109,136110,ASURA,COMPACT,3.5,6.0,not-recorded,Z,not-available,12.77 liters per 100 km,13.94 km/L,230.0,NaN,NaN


as we can see that column that has -9999 / -1 / zero / not-available has information from other column that we can use to find that value with the information that

> column 'Fuel Consumption Comb' is combined fuel consumption rating, calculated as a blend of 55% city driving and 45% highway driving.

So that mean we have the calculation of \
**Combined Fuel Consumption=(City Consumption×0.55t)+(Highway Consumption×0.45)**

But first, that means we need to separate each column to numeric

## Fuel Consumption Column Preprocessing

In [20]:
df_train['numeric_FuelCity'] = df_train['Fuel Consumption City'].str.extract(r'([\d.]+)').astype(float)
df_train['MeasureUnit_FuelCity'] = df_train['Fuel Consumption City'].str.extract(r'\s(.+)')

In [21]:
df_train['numeric_FuelHwy'] = df_train['Fuel Consumption Hwy'].str.extract(r'([\d.]+)').astype(float)
df_train['MeasureUnit_FuelHwy'] = df_train['Fuel Consumption Hwy'].str.extract(r'\s(.+)')

In [22]:
df_train['numeric_FuelComb'] = df_train['Fuel Consumption Comb'].str.extract(r'([\d.]+)').astype(float)
df_train['MeasureUnit_FuelComb'] = df_train['Fuel Consumption Comb'].str.extract(r'\s(.+)')

In [23]:
df_train.sample(5)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity,numeric_FuelHwy,MeasureUnit_FuelHwy,numeric_FuelComb,MeasureUnit_FuelComb
23364,23365,CHEVO,SUV - STANDARD,NaN,8.0,A6,X,1.48 L/10km,13.03 liters per 100 km,16.76 MPG (AS),302.0,1.48,L/10km,13.03,liters per 100 km,16.76,MPG (AS)
66506,66507,JIPU,SUV - SMALL,2.5,not-recorded,A6,X,-9999,34.23 km per L,0.67 L/10km,224.0,9999.00,NaN,34.23,km per L,0.67,L/10km
63367,63368,BMV,SUV - STANDARD,4.4,NaN,AS8,Z,10.02 km per L,11.09 km/L,0.95 L/10km,316.0,10.02,km per L,11.09,km/L,0.95,L/10km
100807,100808,NIRRAN,SUV - SMALL,3.5,6.0,AV,X,1.10 liters per 100 km,80.44 mpg Imp.,NaN,259.0,1.10,liters per 100 km,80.44,mpg Imp.,NaN,NaN
86779,86780,NIRRAN,STATION WAGON - SMALL,NaN,4.0,M6,NaN,31344.85 MPG (AS),75.25 mpg Imp.,166.81 mpg Imp.,104.0,31344.85,MPG (AS),75.25,mpg Imp.,166.81,mpg Imp.


In [25]:
df_train['MeasureUnit_FuelCity'].value_counts()

L/10km               16614
L/100km              16612
km per L             16600
km/L                 16408
L/100 km             16363
mpg Imp.             16308
liters per 100 km    16098
MPG (AS)             16090
Name: MeasureUnit_FuelCity, dtype: int64

In [26]:
df_train['MeasureUnit_FuelHwy'].value_counts()

km/L                 16502
mpg Imp.             16365
MPG (AS)             16365
L/10km               16346
L/100 km             16312
km per L             16299
L/100km              16265
liters per 100 km    16199
Name: MeasureUnit_FuelHwy, dtype: int64

In [27]:
df_train['MeasureUnit_FuelComb'].value_counts()

MPG (AS)             16458
km/L                 16443
L/100 km             16409
L/100km              16357
liters per 100 km    16305
km per L             16278
L/10km               16231
mpg Imp.             16170
Name: MeasureUnit_FuelComb, dtype: int64

So, I want to convert all the measure to km/L

### Equation & Validation

In [46]:
df_checking = df_train.copy()

#### L/10km --> km/L

In [47]:
df_checking['numeric_FuelCity'] = np.where(df_train['MeasureUnit_FuelCity'] == 'L/10km', 10 / df_train['numeric_FuelCity'], df_train['numeric_FuelCity'])
df_checking[df_checking['MeasureUnit_FuelCity'] == 'L/10km'].head(3)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity,numeric_FuelHwy,MeasureUnit_FuelHwy,numeric_FuelComb,MeasureUnit_FuelComb
2,3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,0.17 L/10km,2.03 liters per 100 km,zero,329.0,58.823529,L/10km,2.03,liters per 100 km,NaN,NaN
12,13,TOYOTI,MINIVAN,2.0,4.0,AS6,X,0.22 L/10km,5.30 L/100 km,27.94 km per L,260.0,45.454545,L/10km,5.30,L/100 km,27.94,km per L
26,27,NIRRAN,PICKUP TRUCK - SMALL,NaN,NaN,M6,X,0.53 L/10km,3.58 liters per 100 km,22.11 km per L,307.0,18.867925,L/10km,3.58,liters per 100 km,22.11,km per L


In [44]:
df_train[df_train['MeasureUnit_FuelCity'] == 'L/10km'].head(3)

,Id,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),numeric_FuelCity,MeasureUnit_FuelCity,numeric_FuelHwy,MeasureUnit_FuelHwy,numeric_FuelComb,MeasureUnit_FuelComb
2,3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,0.17 L/10km,2.03 liters per 100 km,zero,329.0,0.17,L/10km,2.03,liters per 100 km,NaN,NaN
12,13,TOYOTI,MINIVAN,2.0,4.0,AS6,X,0.22 L/10km,5.30 L/100 km,27.94 km per L,260.0,0.22,L/10km,5.30,L/100 km,27.94,km per L
26,27,NIRRAN,PICKUP TRUCK - SMALL,NaN,NaN,M6,X,0.53 L/10km,3.58 liters per 100 km,22.11 km per L,307.0,0.53,L/10km,3.58,liters per 100 km,22.11,km per L


#### L/100km --> km/L

In [ ]:
df_checking['numeric_FuelCity'] = np.where(df_train['MeasureUnit_FuelCity'] == 'L/10km', 1 / df_train['numeric_FuelCity'], df_train['numeric_FuelCity'])
df_checking[df_checking['MeasureUnit_FuelCity'] == 'L/10km'].head(3)